# Bulk download fic pdfs

In [50]:
import os
import json
import sys
sys.path.append("..")
import requests
import pandas as pd
from time import time
from multiprocessing.pool import ThreadPool

In [52]:
pwd

'/Users/yuchen.zhang/Documents/Projects/Class_Action/class_action_git/src'

In [51]:
# Load input data
scraped_data = pd.read_csv("../data/data_from_scrapy.csv")

In [44]:
# Example fic summary table
pd.read_json(scraped_data["fic_summary_table"][0])

,No,Document Title,Filing Date,link
0,1,Class Action Complaint,07/07/2020,http://securities.stanford.edu/filings-documen...
1,2,U.S. District Court Civil Docket,07/07/2020,http://securities.stanford.edu/filings-documen...


In [42]:
# Set Output Folder location
output_folder = "/Users/yuchen.zhang/Documents/Projects/Class_Action/class_action_git/data/Case Filings"

In [27]:
# Function to download a pdf from a link
def url_response(url):
    path, url = url
    r = requests.get(url, stream = True)
    
    with open(output_folder + "/" + path,"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 

             # writing one chunk at a time to pdf file 
             if chunk: 
                 pdf.write(chunk)

In [30]:
# Function to bulk download
def downloadall(data):
    #Loop through data (scraped data)
    for x in range(0, len(data)):
        table = pd.read_json(data["fic_summary_table"][x])
        print(x)
        
        #Loop through each document in fic_summary_table
        for i, j in table.iterrows():
            filename = data["docket"][x] + "_" + table["Document Title"][i] + "_" + table["Filing Date"][i].replace("/","")
            print(filename)
            url_response([filename, table["link"][i]])

In [32]:
#downloadall(scraped_data）

0
20-CV-81063_Class Action Complaint_07072020
20-CV-81063_U.S. District Court Civil Docket_07072020
1
20-CV-02886_Class Action Complaint for Violations of the Federal Securities Laws_06302020
20-CV-02886_U.S. District Court Civil Docket_07092020
2
20-CV-04289_Class Action Complaint_06292020
20-CV-04289_U.S. District Court Civil Docket_06302020
3
20-CV-03326_Complaint - Class Action_07072020
20-CV-03326_U.S. District Court Civil Docket_07082020
4
20-CV-02918_Complaint for Violation of Federal Securities Laws_07012020
20-CV-02918_U.S. District Court Civil Docket_07032020
5
20-CV-05156_Class Action Complaint_07062020
20-CV-05156_Class Action Complaint_07072020
20-CV-05156_U.S. District Court Civil Docket_07072020
6
20-CV-04953_Class Action Complaint for Violation of the Federal Securities Laws_06292020
20-CV-04953_U.S. District Court Civil Docket_07012020
7
20-CV-04944_Class Action Complaint for Violation of the Federal Securities Laws_06282020
20-CV-04944_Class Action Complaint for Viola